<a href="https://colab.research.google.com/github/Ali-Fartout/Getting-Ready-For-Project/blob/main/Collaboration_Filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirement

In [1]:
! pip install kaggle
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d prajitdatta/movielens-100k-dataset

100% 4.77M/4.77M [00:00<00:00, 32.9MB/s]



In [5]:
import zipfile
zip_ref = zipfile.ZipFile("/content/movielens-100k-dataset.zip", "r")
zip_ref.extractall()
zip_ref.close()

# First Look

In [6]:
import pandas as pd
import numpy as np
#Load the u.user file into a dataframe
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv('/content/ml-100k/u.user', sep='|', names=u_cols,
 encoding='latin-1')

users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [7]:
#Load the u.item file into a dataframe
i_cols = ['movie_id', 'title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('/content/ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

movies.head()

,movie_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [8]:
#Remove all information except Movie ID and title
movies = movies[['movie_id', 'title']]
#Load the u.data file into a dataframe
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('/content/ml-100k/u.data', sep='\t', names=r_cols,
 encoding='latin-1')

ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
ratings = ratings.drop('timestamp', axis=1)

# Training and test data

In [10]:
#Import the train_test_split function
from sklearn.model_selection import train_test_split

#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = ratings.copy()
y = ratings['user_id']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

In [11]:
#Import the mean_squared_error function
from sklearn.metrics import mean_squared_error

#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [12]:
#Define the baseline model to always return 3.
def baseline(user_id, movie_id):
    return 3.0

In [13]:
#Function to compute the RMSE score obtained on the testing set by a model
def score(cf_model):
    
    #Construct a list of user-movie tuples from the testing dataset
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    
    #Predict the rating for every user-movie tuple
    y_pred = np.array([cf_model(user, movie) for (user, movie) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['rating'])
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)
score(baseline)

1.2488234462885457

In [15]:
#Build the ratings matrix using pivot_table function
r_matrix = X_train.pivot_table(values='rating', index='user_id', columns='movie_id')

r_matrix.head()

(943, 1641)

In [17]:
#User Based Collaborative Filter using Mean Ratings
def cf_user_mean(user_id, movie_id):
#Check if movie_id exists in r_matrix
  if movie_id in r_matrix:
#Compute the mean of all the ratings given to the movie
    mean_rating = r_matrix[movie_id].mean()
  else:
#Default to a rating of 3.0 in the absence of any information
      mean_rating = 3.0
  return mean_rating
#Compute RMSE for the Mean model
score(cf_user_mean)

1.0300824802393536

In [18]:
#Create a dummy ratings matrix with all null values imputed to 0
r_matrix_dummy = r_matrix.copy().fillna(0)
# Import cosine_score
from sklearn.metrics.pairwise import cosine_similarity
#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)
#Convert into pandas dataframe
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index,
columns=r_matrix.index)
cosine_sim.head(10)

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.108361,0.046638,0.029577,0.245753,0.335853,0.344724,0.191582,0.057149,0.251979,0.235385,0.228356,0.304572,0.234445,0.129171,0.309772,0.188809,0.346046,0.100968,0.199532,0.166757,0.236226,0.297992,0.222576,0.220881,0.206157,0.106212,0.237319,0.128437,0.140023,0.065700,0.181805,0.033940,0.011083,0.024420,0.017731,0.145990,0.144916,0.054834,0.075100,...,0.088132,0.135066,0.115868,0.216639,0.248748,0.112567,0.192244,0.243273,0.164886,0.315299,0.057415,0.063335,0.444329,0.150487,0.210744,0.251917,0.054468,0.263734,0.251835,0.098013,0.229718,0.072834,0.065248,0.172213,0.198114,0.157317,0.211386,0.112948,0.273808,0.380418,0.257073,0.069412,0.231643,0.108093,0.176842,0.104799,0.232472,0.051528,0.129555,0.256333
2,0.108361,1.000000,0.057613,0.130237,0.054918,0.190552,0.079399,0.076146,0.167992,0.147376,0.100710,0.087690,0.179865,0.170904,0.336956,0.082856,0.148535,0.134448,0.070199,0.051890,0.144626,0.025755,0.133631,0.103460,0.113986,0.262372,0.181119,0.045874,0.099802,0.178682,0.085454,0.168848,0.114961,0.168932,0.026121,0.081678,0.057475,0.027820,0.167157,0.191937,...,0.025587,0.236144,0.193923,0.212069,0.044199,0.049732,0.242911,0.000000,0.000000,0.074625,0.038895,0.260004,0.093701,0.255186,0.049829,0.309281,0.311490,0.148226,0.078822,0.175618,0.169203,0.000000,0.194673,0.086257,0.032210,0.076731,0.222380,0.320522,0.010844,0.086376,0.136993,0.252887,0.255454,0.285193,0.232751,0.149088,0.102807,0.062386,0.109143,0.107686
3,0.046638,0.057613,1.000000,0.139805,0.000000,0.032485,0.043869,0.080968,0.022263,0.059925,0.025338,0.000000,0.130786,0.027797,0.064841,0.033790,0.033218,0.000000,0.072457,0.041260,0.153512,0.002765,0.022891,0.025828,0.031536,0.130869,0.000000,0.046583,0.097863,0.071040,0.112275,0.106619,0.286920,0.114850,0.199781,0.157950,0.016675,0.038617,0.178786,0.316503,...,0.041200,0.132704,0.046466,0.080419,0.048187,0.037370,0.045587,0.000000,0.011937,0.022697,0.000000,0.231435,0.028872,0.031366,0.000000,0.172285,0.234889,0.054099,0.024630,0.181877,0.009943,0.150564,0.242289,0.039911,0.077796,0.012870,0.011303,0.150248,0.000000,0.011447,0.027402,0.000000,0.175060,0.010343,0.105635,0.019052,0.127099,0.023917,0.060392,0.000000
4,0.029577,0.130237,0.139805,1.000000,0.000000,0.045190,0.088586,0.199526,0.135013,0.026919,0.076747,0.078065,0.078496,0.083910,0.151148,0.014102,0.071625,0.032728,0.046870,0.063944,0.078067,0.073027,0.034276,0.100940,0.042499,0.133143,0.053838,0.120172,0.133272,0.131637,0.000000,0.137936,0.239482,0.107528,0.102025,0.055604,0.155054,0.062450,0.175228,0.163922,...,0.088836,0.160588,0.037572,0.088923,0.079925,0.115111,0.120411,0.000000,0.000000,0.084925,0.000000,0.122894,0.062253,0.000000,0.000000,0.133174,0.081929,0.109787,0.134332,0.126175,0.066107,0.110519,0.232643,0.121914,0.027958,0.018500,0.073114,0.151860,0.011766,0.069419,0.055392,0.049773,0.076549,0.139382,0.113886,0.000000,0.130343,0.077357,0.157890,0.063911
5,0.245753,0.054918,0.000000,0.000000,1.000000,0.176443,0.281860,0.132205,0.038790,0.134200,0.251593,0.092854,0.248074,0.223169,0.025404,0.228500,0.068526,0.231632,0.103015,0.198412,0.132421,0.307520,0.308427,0.106202,0.218025,0.060633,0.083527,0.299602,0.051691,0.094068,0.055939,0.056472,0.000000,0.003370,0.011725,0.000000,0.091807,0.189470,0.000000,0.000000,...,0.035732,0.030199,0.072539,0.114966,0.177732,0.037206,0.036854,0.220582,0.112036,0.252346,0.023278,0.000000,0.243014,0.069952,0.132249,0.134720,0.000000,0.183636,0.307767,0.074069,0.174632,0.016075,0.000000,0.185434,0.079520,0.199720,0.100829,0.020942,0.237812,0.287200,0.183969,0.019305,0.073714,0.041807,0.081088,0.029743,0.188392,0.068342,0.055557,0.20725